#Section 1

##IMPORT

In [ ]:
!pip3 install ftfy

import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")
import ftfy
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re

from math import exp
from numpy import sign

from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer
!pip install transformers

from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install wordcloud
#from wordcloud import WordCloud

##CONSTANTS

In [ ]:
# Reproducibility
np.random.seed(1234)

DEPRES_NROWS = 3200  # number of rows to read from DEPRESSIVE_TWEETS_CSV
RANDOM_NROWS = 12000 # number of rows to read from RANDOM_TWEETS_CSV
MAX_SEQUENCE_LENGTH = 140 # Max tweet size
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS= 10

LOAD DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#FILE PATHS
#DEPRESSIVE_TWEETS_CSV = '/content/drive/MyDrive/Colab Notebooks/DATA SOURCE 1/dataset/depressive_tweets_processed.csv'
#RANDOM_TWEETS_CSV = '/content/drive/MyDrive/Colab Notebooks/DATA SOURCE 1/dataset/Sentiment Analysis Dataset.csv'
#EMBEDDING_FILE = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'

In [ ]:
#word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/archive 2/DatasetBaru.csv')

In [ ]:
#depressive_tweets_df = pd.read_csv(DEPRESSIVE_TWEETS_CSV, sep = '|', header = None, usecols = range(0,9), nrows = DEPRES_NROWS)
#random_tweets_df = pd.read_csv(RANDOM_TWEETS_CSV, encoding = "ISO-8859-1", usecols = range(0,4), nrows = RANDOM_NROWS)

In [ ]:
#df[df['Sentiment'] == 0]['Sentiment'].count()
#df[df['Sentiment'] == 1]['Sentiment'].count()
#df[df['Sentiment'] == 2]['Sentiment'].count()
df[df['Sentiment'] == 3]['Sentiment'].count()

0

In [ ]:
df

,Unnamed: 0,Sentiment,SentimentText,Unnamed: 0.1,Unnamed: 0.1.1
0,0,negative,Pak PM survives removal scare,NaN,NaN
1,1,negative,but court orders further probe into corruptio...,NaN,NaN
2,2,negative,Supreme Court quashes criminal complaint again...,NaN,NaN
3,3,negative,Art of Living's fights back over Yamuna floodp...,NaN,NaN
4,4,negative,livid.,NaN,NaN
...,...,...,...,...,...
9127,9127,negative,class till muchhhhhh laterrrrr,522551.0,522551.0
9128,9128,negative,@minxlj owwy Ulcers are the worst - I can't e...,283921.0,283921.0
9129,9129,negative,@superxmaika hahah can totally relate to the s...,653273.0,653273.0
9130,9130,negative,"Okay I just have to do this: 300th UPDATE, WOO...",740191.0,740191.0


#Section 2

##Preprocessing

Preprocessing the tweets in order to:



*   Remove links and images
*   Remove hashtags
*   Remove @ mentions
*Remove emojis
*Remove stop words
*Remove punctuation
*Get rid of stuff like "what's" and making it "what is'
*Stem words so they are all the same tense (e.g. ran -> run)


In [ ]:
# Expand Contraction
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
def clean_tweets(tweets, sentiments):
    cleaned_tweets = []
    cleaned_sentiments = []
    for i, tweet in enumerate(tweets):
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            #remove hashtag, @mention, emoji and image URLs
            tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.>)|(pic\.twitter\.com\/.)", " ", tweet).split())

            #fix weirdly encoded texts
            tweet = ftfy.fix_text(tweet)
            tweet = tweet.lower() #case folding
            #expand contraction
            tweet = expandContractions(tweet)

            #remove punctuation
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            #stop words
            #stop_words = set(stopwords.words('english'))
            #word_tokens = nltk.word_tokenize(tweet)
            #filtered_sentence = [w for w in word_tokens if not w in stop_words]
            #tweet = ' '.join(filtered_sentence)

            #stemming words
            #tweet = PorterStemmer().stem(tweet)

            cleaned_tweets.append(tweet)
            cleaned_sentiments.append(sentiments[i])

    return cleaned_tweets, cleaned_sentiments

In [ ]:
# merubah type data dalam record menjadi format string
df.SentimentText=df.SentimentText.astype(str)
#@title
# merubah type data dalam record sentimentText menjadi format string
#depressive_tweets_df=depressive_tweets_df.astype(str)

In [ ]:
#depressive_tweets_arr = [x for x in depressive_tweets_df[5]]
random_tweets_arr = [x for x in df['SentimentText']]
#X_d = clean_tweets(depressive_tweets_arr)
X_r, y_r = clean_tweets(random_tweets_arr, df['Sentiment'])

##Splitting Data

In [ ]:
# defining feature matrix(X) and response vector(y)
X = X_r
#X = [np.array(d) for d in X]
y = y_r
y = np.array(y)
#X = np.array(X)

In [ ]:
# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=1)

In [ ]:
#tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
#tokenizer.fit_on_texts(X_r)


In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
X_train

['i miss my baby unhappy',
 'may i remind you pips depression is not a joke and it will never be a joke',
 'in memoriam of alea starbridge unhappy your beautiful voice will never be forgotten',
 'white lady not good not in crisis mode or anything just yet but this weather is not helping my depression',
 'emoji raised hand medium skin tone hi i am a high functioning person living with depression anxiety ptsd and probably some other undiagnosed stuff as a mexican american woman i always knew the only way i d survive would be to be the best since being mediocre was not an option https twitter com franciscolwhite status 988770953286086656',
 'lmaooo i posted my story on instagram and it mentions my depression one chick that does not follow me liked it but the gag is she harassed me and threatened to beat me up over a party freshman year contributing to my depression emoji face with tears of joy',
 'depression levels 14554550 https twitter com sonlarruyalar status 989208187231461376',
 'fro

In [ ]:
X_test

['best options for the depression treatment http bit ly 2qzsnlc',
 'new game guess the word it is a 5 letter noun hint n related words are scallop dent dimple depression tooth battlement',
 'word of the day is depression',
 '22 walker mn thomas123 depression',
 'girl in red summer depression lyrics https youtu be mgipust7bxw via',
 'thanks for sharing happy',
 'former captain michael john clarke on strengths of a team',
 'causes of insomnia regret self blame overthinking anger depressionlonelinessand her',
 'economic blockade will not impact process',
 't1 early intervention also gp aware of impact of physical health on psychological well being aside from discrete mh difficulties ie anxiety depression',
 'how come i did not get a reminder email this month unhappy',
 'wowwow these are all so cute unhappy i wantt',
 'there are times depression will hit you out of nowhere even you are enjoying your day',
 'excited about event next week in bangalore come happy',
 'my memes are ironic my de

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
max_seq = 200
lr = 2e-5
epochs = 20
batch_size = 16

In [ ]:
def batch_encode(X, tokenizer):
  return tokenizer.batch_encode_plus(X,
                                     max_length=max_seq, # set the length of the sequences
                                     add_special_tokens=True, # add [CLS] and [SEP] tokens
                                     return_attention_mask=True,
                                     return_token_type_ids=False, # not needed for this type of ML task
                                     padding='max_length', # add 0 pad tokens to the sequences less than max_length
                                     return_tensors='tf')

In [ ]:
X_train = (batch_encode(X_train, tokenizer)).values()
X_test = (batch_encode(X_test, tokenizer)).values()

In [ ]:
#nb_words = min(MAX_NB_WORDS, len(word_index))

#embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

#for (word, idx) in word_index.items():
#    if word in word2vec.vocab and idx < MAX_NB_WORDS:
#        embedding_matrix[idx] = word2vec.word_vec(word)

In [ ]:
#nb_words

In [ ]:
len(y_r)

8908

In [ ]:
from sklearn.preprocessing import OneHotEncoder
y = pd.get_dummies(y_train)
y_train = np.array(y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
y = pd.get_dummies(y_test)
y_test = np.array(y)

In [ ]:
X_train

dict_values([<tf.Tensor: shape=(7126, 200), dtype=int32, numpy=
array([[  101,  1045,  3335, ...,     0,     0,     0],
       [  101,  2089,  1045, ...,     0,     0,     0],
       [  101,  1999, 24443, ...,     0,     0,     0],
       ...,
       [  101,  2013, 13075, ...,     0,     0,     0],
       [  101, 21541,  2290, ...,     0,     0,     0],
       [  101,  1037,  2000, ...,     0,     0,     0]], dtype=int32)>, <tf.Tensor: shape=(7126, 200), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>])

In [ ]:
y_train

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]], dtype=uint8)

In [ ]:
#tes = model([X_train['input_ids'], X_train['attention_mask']])

#Model

In [ ]:
import tensorflow as tf
import keras

In [ ]:
!pip install -q keras-rectified-adam
from keras_radam import RAdam

In [ ]:
input_word_ids = tf.keras.layers.Input(shape=(max_seq,), name='input_word_ids', dtype='int32')
input_mask_ids = tf.keras.layers.Input(shape=(max_seq,), name='input_mask_ids', dtype='int32')
bert = bert_model([input_word_ids, input_mask_ids])
sequence_output = bert['last_hidden_state']

In [ ]:
input_word_ids = tf.keras.layers.Input(shape=(max_seq,), name='input_word_ids', dtype='int32')
input_mask_ids = tf.keras.layers.Input(shape=(max_seq,), name='input_mask_ids', dtype='int32')
bert = bert_model([input_word_ids, input_mask_ids])
sequence_output = bert['last_hidden_state']
dropout = tf.keras.layers.Dropout(0.2)(sequence_output)
lstm = tf.keras.layers.LSTM(768, return_sequences=True)(dropout)
dropout = tf.keras.layers.Dropout(0.2)(lstm)
pooling = tf.keras.layers.GlobalAveragePooling1D()(dropout)
hidden = tf.keras.layers.Dense(150, activation='relu')(pooling)
outputs = tf.keras.layers.Dense(4, activation='softmax')(hidden)
model = tf.keras.models.Model(inputs = [input_word_ids,input_mask_ids], outputs = outputs)
model.compile(
  optimizer=RAdam(learning_rate=lr),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)
# early_stop = EarlyStopping(monitor='val_accuracy', mode ='max', verbose=1, patience=5,restore_best_weights=True) #Stop training when a monitored metric has stopped improving.

# hist = model.fit(X_train, y_train,
#         validation_data=(X_test, y_test),
#         epochs=20, batch_size=32,
#         callbacks=early_stop)

In [ ]:
model.compile(
  optimizer=RAdam(learning_rate=lr),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

In [ ]:
print(tf.keras.utils.plot_model)
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import classification_report

<function plot_model at 0x7f8d358ec830>


In [ ]:
def model_evaluate():
  y_pred = model.predict(X_test)
  plt.figure(3)
  cf_matrix = ConfusionMatrixDisplay.from_predictions(y_test.argmax(axis=1), y_pred.argmax(axis=1), cmap='Blues')
  plt.title('Confusion Matrix')
  plt.savefig("/content/drive/MyDrive/CSV Skripsi/BERT/BERT Skenario-16-DropoutLayer-{}.png".format(i))
  plt.show()
def model_confusion():
  from sklearn.metrics import ConfusionMatrixDisplay
  from sklearn.metrics import confusion_matrix
  import pandas as pd
  from sklearn.metrics import classification_report
  y_pred = model.predict(X_test)
  report_dict = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), digits=3, output_dict = True)
  report = pd.DataFrame(report_dict)
  report.to_csv("/content/drive/MyDrive/CSV Skripsi/BERT/BERT Skenario-16-DropoutLayer-{}.csv".format(i), index=False)
  print(report)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 200)]        0           []                               
                                                                                                  
 input_mask_ids (InputLayer)    [(None, 200)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_mask_ids[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 200,                                           

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', mode = 'max', verbose = 1, patience=3, restore_best_weights= True) #Stop training when a monitored metric has stopped improving.
for i in range(5):
  hist = model.fit(X_train, y_train, \
                   validation_data=(X_test, y_test), \
                   epochs=20, batch_size=16, shuffle=True,
                   callbacks = early_stop)
  model_evaluate()
  model_confusion()
 # model.save(checkpoint_path, 'saved_model')
  print('--------\n\n')

Epoch 1/20


135/446 [========>.....................] - ETA: 2:15:07 - loss: 1.0214 - accuracy: 0.5532

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Checkpoint Farhan/"
callbacks_list = [ModelCheckpoint(checkpoint_path+"BERT New",
                                  monitor='val_accuracy',
                                  save_best_only=True,
                                  mode='max',
                                  verbose=1)]

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Checkpoint Farhan/"
model.load_weights('/content/drive/MyDrive/Checkpoint Farhan/BERT + Regularizer')

In [ ]:
#model = Sequential()
# Embedded layer
#model.add(Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix],
#                            input_length=MAX_SEQUENCE_LENGTH, trainable=False))
# Convolutional Layer
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Dropout(0.2))
# LSTM Layer
#model.add(LSTM(300))
#model.add(Dropout(0.2))
#model.add(Dense(3, activation='softmax'))


In [ ]:
print(type(y))

In [ ]:
print(tf.keras.utils.plot_model)

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
report_dict = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), digits=3, output_dict = True)
pd.DataFrame(report_dict)

In [ ]:
plt.figure(3)
cf_matrix = ConfusionMatrixDisplay.from_predictions(y_test.argmax(axis=1), y_pred.argmax(axis=1), cmap='Blues')
plt.title('Confusion Matrix')
plt.savefig(checkpoint_path+'BERT New -LSTM Confusion Matrix.png',dpi=300,bbox_inches='tight')
plt.show()